In [ ]:
import numpy as np
import pandas as pd
import random

# Creation of a synthetic example

In [ ]:
def create_synthetic_attention_data_varying_lengths(num_samples, target_range, source_range):
    data = []
    for _ in range(num_samples):
        target_len = random.randint(*target_range)
        source_len = random.randint(*source_range)
        attn_matrix = np.random.rand(target_len, source_len)
        attn_matrix /= attn_matrix.sum()
        mt_len = target_len
        data.append({"idx": len(data), "attn": attn_matrix, "mt_len": mt_len})
    return pd.DataFrame(data)

dataset_stats = create_synthetic_attention_data_varying_lengths(1000, (10, 30), (10, 30))
dataset_reference_stats = create_synthetic_attention_data_varying_lengths(100000, (10, 30), (10, 30))

# Save the datasets to files
dataset_stats.to_pickle("dataset_stats_varying_lengths.pkl")
dataset_reference_stats.to_pickle("dataset_reference_stats_varying_lengths.pkl")

In [ ]:
from utils import wass_dist_computation
import argparse

args = argparse.Namespace(
    dataset_stats_path="dataset_stats_varying_lengths.pkl",
    dataset_reference_stats_path="dataset_reference_stats_varying_lengths.pkl",
    metric_space="uni",
    length_window=10.0,
    compare_to_uni=True,
    no_samples=100,
    bottom_k=4,
    seed=38
)

dataset_stats = pd.read_pickle(args.dataset_stats_path)
dataset_reference_stats = pd.read_pickle(args.dataset_reference_stats_path)

wasserstein_distances = wass_dist_computation(dataset_stats, dataset_reference_stats, args)
# for wass-to-data (compare_to_uni=False), the shape of wasserstein_distances should be [len(dataset_stats), no_samples]; you can aggregate via bottom-k
# e.g., wasserstein_distances = [np.mean(sorted(x_samp)[:args.bottom_k]) for x_samp in wasserstein_distances]

# for wass-to-uni (compare_to_uni=True), the shape of wasserstein_distances should be [len(dataset_stats)]. No need to aggregate.

# Replicate our results

In [ ]:
dataset_stats = pd.read_pickle("/home/nunomg/ot-hallucination-detection/hallucinations_deen_w_stats_and_scores.pkl")
dataset_attn_reference = pd.read_pickle("/home/nunomg/ot-hallucination-detection/reference_set_deen.pkl")


In [ ]:
# Replicate Wass-to-Data
args = argparse.Namespace(
    dataset_stats_path="dataset_stats_varying_lengths.pkl",
    dataset_reference_stats_path="dataset_reference_stats_varying_lengths.pkl",
    metric_space="l1",
    length_window=10.0,
    compare_to_uni=False,
    no_samples=1000,
    bottom_k=4,
    seed=38
)

# Replicate Wass-to-Uni
args = argparse.Namespace(
    dataset_stats_path="dataset_stats_varying_lengths.pkl",
    dataset_reference_stats_path="dataset_reference_stats_varying_lengths.pkl",
    metric_space="uni",
    length_window=10.0,
    compare_to_uni=False,
    no_samples=1000,
    bottom_k=4,
    seed=38
)    

wasserstein_distances = wass_dist_computation(dataset_stats, dataset_attn_reference, args)
# compute metrics 
from utils import compute_detection_metrics_wass
dataset_stats["wass_dist"] = wasserstein_distances
auroc_metrics = compute_detection_metrics_wass(wasserstein_distances=wasserstein_distances, dataset_stats=dataset_stats, args=args, bottom_k=args.bottom_k) # compute detection metrics